In [ ]:
!pip install scratchai-nightly  # for adversarial attack
!pip install torchvision==0.9.1 # deep learning models
!pip install flashtorch         # visualization based on activation maximization 
!pip install mapextrackt        # visualization of neural network saliency map

In [ ]:
# download and store locally a stop sign image
stop_sign_url = 'https://static01.nyt.com/images/2011/12/11/magazine/11wmt1/mag-11WMT-t_CA0-jumbo.jpg'
!mkdir input_images
# !wget https://static01.nyt.com/images/2011/12/11/magazine/11wmt1/mag-11WMT-t_CA0-jumbo.jpg -O input_images/stop.jpg
!wget https://github.com/trust-ai/SafeBench/blob/main/safebench/scenario/scenario_data/template_od/stopsign.jpg -O input_images/stop.jpg

In [ ]:
# suppress error
import logging as logging
import sys as sys
logging.disable(sys.maxsize) 

# import the library
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision import models
from scratchai import *

from flashtorch.activmax import GradientAscent
from MapExtrackt import FeatureExtractor
from torch.distributions import Normal


# set parameters
stop_sign_path = 'input_images/stop.jpg' #stop sign image path
true_class = 919 # imagenet id for street sign


# function handle to get prediction more easily
def get_prediction(image, model):
    #assumes img and net are datasets and models trained using imagenet dataset  
    confidences = model(image.unsqueeze(0))
    class_idx = torch.argmax(confidences, dim=1).item()
    class_label = datasets.labels.imagenet_labels[class_idx]
    return class_label, confidences[0, class_idx].item(), class_idx

**1a: Making prediction**

In [ ]:
# load and preprocess the stop sign image
img = imgutils.load_img(stop_sign_path)
img = imgutils.get_trf('rz256_cc224_tt_normimgnet')(img) #normalize and reshape the input image

# REPLACE THE THREE DOTS WITH YOUR OWN CODE

net = ...  # load resnet

# use the provided get_prediction function to predict the class of the stop sign image
...

**1b: Random perturbation**

In [ ]:
# REPLACE THE THREE DOTS WITH YOUR OWN CODE

epsilon = ... # set the epsilon

torch.manual_seed(0) # set the random seed when you use functions that uses sampling

noisy_img = ... # perform uniform random attack here [see the example in the Sec. 3.2.4 of the problem set]

... # output prediction, conf, and label_id using get_prediction function

imgutils.imshow([img, noisy_img-img, noisy_img], normd=True) #output the original image, the perturbation image, the perturbed image

**1c: FGM Attack**

In [ ]:
# REPLACE THE THREE DOTS WITH YOUR OWN CODE

images, true_labels, predicted_labels = ... # perform FGM attacks and return all the outputs

imgutils.imshow(images) # show all the images [original, perturbation, and adversarial]

true_labels, predicted_labels # show true and predicted labels

**1d: PGD Attack**

In [ ]:
# REPLACE THE THREE DOTS WITH YOUR OWN CODE

target_class = 829 # imagenet id for street car

images, true_labels, predicted_labels = ... # perform PGD attacks and return all the outputs

imgutils.imshow(images) # show all the images [original, perturbation, and adversarial]

true_labels, predicted_labels #show true and predicted labels

**2a: AlexNet layer 0 visualization**

In [ ]:
# REPLACE THE THREE DOTS WITH YOUR OWN CODE

model = ...  #load pretrained alexnet

print(model) #show the alexnet structure

In [ ]:
# REPLACE THE THREE DOTS WITH YOUR OWN CODE

#load GradientAscent on GPU
g_ascent = GradientAscent(model.features)
g_ascent.use_gpu = True

layer_idx = ... # set the layer index

filters = ... # set the filter numbers

layer = model.features[layer_idx] # select the 1st conv layer

... # call g_ascent.visualize() with the correct arguments to output the visualization



**2b: AlexNet layer 10 visualization**

In [ ]:
# MODIFY THE CODE FOR 2a TO VISUALIZE LAYER 10, FILTERS [5, 10, 15, 20] OF ALEXNET


**2c: AlexNet saliency map with the stop sign image**

In [ ]:
# REPLACE THE THREE DOTS WITH YOUR OWN CODE

# load FeatureExtractor
from MapExtrackt import FeatureExtractor

layer_idx = ... #define the layer index

fe = FeatureExtractor(model)

fe.set_image(stop_sign_path) # stop_sign_path is the path to the stop sign image

fe.display_from_map(layer_no=layer_idx)

**3: Example of plot with a confidence interval**



In [ ]:
#THIS IS JUST AN EXAMPLE TO PLOT CONFIDENCE INTERVAL AS SHADED AREA

n = 500 # number of samples
k = 10  # number of replications
sigma = 0.2

torch.manual_seed(0) # set the random seed
deltas = torch.FloatTensor(sigma*torch.randn(n, k)) # gaussian samples ~ N(0, sigma*I)

# compute mean and standard deviation
mean_ = deltas.mean(dim=0)
std_ = deltas.std(dim=0)

# generate the plot
x = np.arange(k) # populate x axis
plt.plot(x, mean_, label="Estimated mean of $\delta$")
plt.fill_between(x, mean_ - std_, mean_ + std_, alpha=0.5, label="Confidence Interval") # 1-sigma confidence interval
plt.legend()
plt.ylim([None, 0.4]) 
plt.title('Example plot')
plt.show()

**Density computation example with log_prob**

In [ ]:
# THIS IS AN EXAMPLE TO USE log_prob METHOD FOR EASIER DENSITY COMPUTATION

# Suppose you want to compute the density of Normal distribution

# create Normal distribution object
p = Normal(torch.tensor([0.0]), torch.tensor([sigma]))       # N(0, sigma**2)
p_tilde = Normal(torch.tensor([0.2]), torch.tensor([sigma])) # N(1, sigma**2)

# use log_prob method
log_density_orig = p.log_prob(deltas) # log_prob method gives you log densities
log_density_tilde = p_tilde.log_prob(deltas) 

# verify this by plotting the density, i.e. the exp of the log_density
plt.scatter(deltas, torch.exp(log_lkl_orig), label="p")
plt.scatter(deltas, torch.exp(log_lkl_tilde), label="p_tilde")
plt.show()

**3a: MC estimator for prob. robustness of ResNet-18**

In [ ]:
# REPLACE THE THREE DOTS WITH YOUR OWN CODE

net = ... # load resnet

sigma = # parameter sigma

# evaluate the model k times, each time use n samples
k = ... # number of replications
n = ... # number of samples in each replication

# collect the samples
torch.manual_seed(0) # set the random seed
deltas = torch.FloatTensor(sigma*torch.randn(*img.shape, n, k)) # gaussian samples ~ N(0, sigma*I)

resnet_test=np.zeros([n, k])

for i in range(k):
    for j in range(n):
        noisy_img = img + deltas[:, :, :, j, i]
        _, _, y_i = get_prediction(noisy_img, net)
        resnet_test[j, i] = y_i != true_class

In [ ]:
# compute the mean and standard deviation of your estimator
mu_hat_n_samples = resnet_test.mean(axis=0)

mean_ = mu_hat_n_samples.mean()
std_ = mu_hat_n_samples.std()

mean_, std_

In [ ]:
# REPEAT THE ABOVE EXAMPLE FOR n = [100, ..., 1000]
...

# PLOT THE MEAN AND THE CONFIDENCE INTERVAL OF THE k VALUES OF mu_hat_n VS n
...

**3b: MC relative error**

In [ ]:
# REPLACE THE THREE DOTS WITH YOUR OWN CODE

mu = 0.03 #true mu value

relative_error = ... # compute the relative error

# PLOT THE RELATIVE ERROR VS n
...

**3c: Misclassification rate w.r.t. samples close to an adversarial example**

In [ ]:
# REPLACE THE THREE DOTS WITH YOUR OWN CODE

# repeat the FGM attack from problem 1c
xtilde = ...

# perform mean shift, closer to the adversarial example
scale = 1/3
mean_shift = scale*xtilde

# generate Gaussian samples centered at the mean_shift
torch.manual_seed(0) # set the random seed

deltas = ...

# compute misclassification rate using this new deltas, similar to 3a above
...

**3d: IS estimator**

In [ ]:
# REPLACE THE THREE DOTS WITH YOUR OWN CODE

# use the sample generated in 3c and the log_prob method to compute the likelihood ratio
p = Normal(torch.tensor([0.0]), torch.tensor([sigma])) # N(0, sigma**2)

log_density_p = p.log_prob(deltas) # log_density per dimension for each sample under dist p = N(0, sigma**2)

log_density_ptilde = p.log_prob(deltas - mean_shift) # log_density per dimension for each sample under dist ptilde = N(mean_shift, sigma**2)

# compute log_likelihood per sample (assuming i.i.d. noise )
log_lkl_p = log_density_p.view(-1, log_density_p.size(3), log_density_p.size(4)).sum(axis=0)
log_lkl_ptilde = log_density_ptilde.view(-1, log_density_ptilde.size(3), log_density_ptilde.size(4)).sum(axis=0)

# compute likelihood ratio
lkl_ratio = torch.exp(log_lkl_p - log_lkl_ptilde)

# proceed to compute the IS estimator
...


In [ ]:
# similar to 3a, repeat for n = [100, ..., 1000]
...

# plot the mean and the confidence interval of mu_hat_n vs n
...

**3e: IS relative error**

In [ ]:
# similar to 3b, but use the IS result
...
